In [ ]:
import pandas as pd

In [ ]:
# Load the Excel file into a pandas DataFrame
file_path = 'model_output_Oshot.xlsx'
df = pd.read_excel(file_path)

In [ ]:
# Determine the range for the Abstract Number
range_end = min(len(df), 100)

# Add the Abstract Number column
df['Abstract Number'] = range(1, range_end + 1)

In [ ]:
df.head()

,Model Output,Abstract Number
0,Copper toxicosis: Category 1 - Modifier\nSpeci...,1
1,Entities:\n* Modifier: &quot;denaturing gradie...,2
2,Entities:\n\n* Modifiers: 4 (category 1)\n\t+ ...,3
3,Entities:\n\n* CUG-BP (Category 1: Modifier)\n...,4
4,Entities:\n* Modifiers: none\n* Specific disea...,5


In [ ]:
# Function to preprocess each row according to the specified modifications
def preprocess_text(row):
    # Split the text into lines
    lines = row['Model Output'].split('\n')
    new_rows = []

    for line in lines:
        # Replace bullet points (*) with '#' and handle sub-points
        if line.startswith('\t+'):
            line = line.replace('\t+', '    +')  # Replace tab with spaces for sub-points
        elif line.startswith('*'):
            line = line.replace('*', '#')

        # Create a new row for each line, maintaining the same abstract number
        new_rows.append({'Model Output': line, 'Abstract Number': row['Abstract Number']})

    return new_rows

In [ ]:
# Apply the preprocessing to each row and concatenate the results
preprocessed_rows = [preprocess_text(row) for index, row in df.iterrows()]
preprocessed_df = pd.DataFrame([item for sublist in preprocessed_rows for item in sublist])

In [ ]:
preprocessed_df.head(10)  # Displaying the first 10 rows of the new dataframe

,Model Output,Abstract Number
0,Copper toxicosis: Category 1 - Modifier,1
1,Specific Disease: Non-Indian childhood cirrhosis,1
2,Disease class: Copper metabolism disorders,1
3,Entities:,2
4,# Modifier: &quot;denaturing gradient gel elec...,2
5,# Specific Disease: &quot;familial adenomatous...,2
6,# Disease Class: &quot;cancer&quot;,2
7,Entities:,3
8,,3
9,# Modifiers: 4 (category 1),3


In [ ]:
def remove_html_quotes(text):
    # Remove "&quot;" from the string
    return text.replace('&quot;', '')

# Apply the function to remove "&quot;" from each row in the 'Model Output' column
preprocessed_df['Model Output'] = preprocessed_df['Model Output'].apply(remove_html_quotes)

In [ ]:
#Removing the '#' character from each row in the 'Model Output' column
preprocessed_df['Model Output'] = preprocessed_df['Model Output'].str.replace('#', '')

In [ ]:
# Function to merge rows based on the '+' character
def merge_rows_with_plus(df):
    merged_list = []  # To store the merged rows
    previous_row = None

    for index, row in df.iterrows():
        # Check if the current row contains a '+' character
        if '+' in str(row['Model Output']):
            if previous_row is not None and '+' in str(previous_row['Model Output']):
                # Merge with previous row
                merged_row = previous_row['Model Output'] + ', ' + row['Model Output']
                previous_row = row
                previous_row['Model Output'] = merged_row
            else:
                # If the previous row does not contain '+', simply add the current row to the list
                if previous_row is not None:
                    merged_list.append(previous_row)
                previous_row = row
        else:
            # If the current row does not contain '+', add the previous row (if any) and the current row to the list
            if previous_row is not None:
                merged_list.append(previous_row)
                previous_row = None
            merged_list.append(row)

    # Add the last row if it hasn't been added yet
    if previous_row is not None:
        merged_list.append(previous_row)

    # Create a new DataFrame from the merged list
    return pd.DataFrame(merged_list)

# Apply the function to the DataFrame
df_modified = merge_rows_with_plus(preprocessed_df)

In [ ]:
# Removing any numbers and the strings "category" or "categories" from each row in the 'Model Output' column
df_modified['Model Output'] = df_modified['Model Output'].str.replace(r'\bcategory\b|\bcategories\b|\d+', '', regex=True)
df_modified['Model Output'] = df_modified['Model Output'].str.replace('category', '', case=False, regex=False)

In [ ]:
# Function to merge rows that start with the '+' character with the row above
def merge_plus_rows(df):
    merged_list = []  # To store the merged rows
    previous_row = None

    for index, row in df.iterrows():
        # Check if the current row starts with a '+' character
        if str(row['Model Output']).strip().startswith('+'):
            if previous_row is not None:
                # Merge with previous row
                merged_row = previous_row['Model Output'] + ' ' + row['Model Output']
                previous_row['Model Output'] = merged_row
            else:
                # If there is no previous row, just add the current row to the list
                merged_list.append(row)
        else:
            # Add the previous row to the list and set the current row as the new previous row
            if previous_row is not None:
                merged_list.append(previous_row)
            previous_row = row

    # Add the last row if it hasn't been added yet
    if previous_row is not None:
        merged_list.append(previous_row)

    # Create a new DataFrame from the merged list
    return pd.DataFrame(merged_list)

# Apply the function to the DataFrame
df_merged = merge_plus_rows(df_modified)

Now here we will do the classfication

In [ ]:
# List of strings to search for
search_strings = ["Modifier", "Specific Disease", "Specific Diseases",
                  "SpecificDisease", "Disease class", "Disease Classes", "Disease Class", "Specific disease"]

# Function to extract and remove the specified strings from 'Model Output'
def extract_and_remove_strings(row):
    entity_class = None
    model_output = str(row['Model Output'])

    for string in search_strings:
        if string in model_output:
            entity_class = string
            model_output = model_output.replace(string, '').strip()
            break

    return pd.Series([model_output, entity_class])

# Apply the function to each row and create new columns
df_merged[['Model Output', 'Entity_class']] = df_merged.apply(extract_and_remove_strings, axis=1)


In [ ]:
# Removing rows where 'Model Output' is null or contains only "Entities"
df_merged = df_merged[~df_merged['Model Output'].isna() &
                              ~df_merged['Model Output'].str.strip().eq('Entities:')]
# Removing specific characters and strings from all rows in 'Model Output'
characters_to_remove = ["s:", "+", ":", "es:", "()", "(:)"]
for char in characters_to_remove:
    df_merged['Model Output'] = df_merged['Model Output'].str.replace(char, '', regex=False)

<ipython-input-12-d21a4c4014db>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Model Output'] = df_merged['Model Output'].str.replace(char, '', regex=False)


In [ ]:
# Filtering the data for Abstract Number 3
df_merged_abstract_3 = df_merged[df_merged['Abstract Number'] == 3]

# Extracting and splitting the 'Model Output' entries
df_merged_abstract_3_model_output = df_merged_abstract_3['Model Output'].dropna()
split_model_output = df_merged_abstract_3_model_output.apply(lambda x: '\n'.join(x.split(',')))

In [ ]:
#SAVING THE FILE
output_file_path = 'modified_output.xlsx'

# Save the dataframe to an Excel file
split_model_output.to_excel(output_file_path, index=False)